In [35]:
import pandas as pd  
import numpy as np

In [4]:
df_bridges = pd.read_excel("../data/raw/BMMS_overview.xlsx")
print(df_bridges.columns)
print(df_bridges.head(3))
print("rows:", len(df_bridges))

Index(['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition',
       'structureNr', 'roadName', 'chainage', 'width', 'constructionYear',
       'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon',
       'EstimatedLoc'],
      dtype='object')
  road     km              type  LRPName               name  length condition  \
0   N1  1.800       Box Culvert  LRP001a                  .   11.30         A   
1   N1  4.925       Box Culvert  LRP004b                  .    6.60         A   
2   N1  8.976  PC Girder Bridge  LRP008b  Kanch pur Bridge.  394.23         A   

   structureNr                                           roadName chainage  \
0       117861  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...      1.8   
1       117862  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    4.925   
2       119889  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    8.976   

   width  constructionYear  spans   zone circle     division   sub-division  \
0   19.5       

In [5]:
df_bridges.head()

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.5,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.4,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise
3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.88,12.2,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699833,90.530722,bcs1
4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.2,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699667,90.530722,bcs1


In [6]:
df[["km","lat","lon"]].describe()

,km,lat,lon
count,21406.000000,21313.000000,21313.000000
mean,41.217405,23.828505,90.227328
std,78.810955,2.247895,3.364169
min,0.000000,0.000000,0.000000
25%,6.207250,22.942527,89.377806
50%,15.147000,23.823743,90.289194
75%,35.012250,24.720939,91.288417
max,522.718000,91.544194,93.298416


In [ ]:
#dit is eig cluster 1: de bridges met sws verkeerde waardes 
# 1) Missing coords (NAN values)
mask_missing = df_bridges["lat"].isna() | df_bridges["lon"].isna() 
#--> oplossing: kijken in de andere data sets of het daar wel in staat? je weet wel weg waar op zou staan

# 2) Zero coords (0 or 0.0)
mask_zero = (df_bridges["lat"] == 0) | (df_bridges["lon"] == 0) 


# 3) Invalid range coords
mask_invalid = (df["lat"] < -90) | (df["lat"] > 90) | (df["lon"] < -180) | (df["lon"] > 180)

print("Missing:", mask_missing.sum())
print("Zero:", mask_zero.sum())
print("Invalid:", mask_invalid.sum())

# (optioneel) laat voorbeelden zien
df.loc[mask_invalid, ["road", "km", "lat", "lon"]].head(10) #dit zijn maar kleine fouten net boven 91 graden niet te veel aandacht aan? 
df.loc[mask_zero, ['']

Missing: 94
Zero: 17
Invalid: 15


,road,km,lat,lon
2864,R241,29.840,91.543889,24.805111
2865,R241,31.212,91.538722,24.816556
2866,R241,32.713,91.530556,24.828056
2867,R241,33.126,91.527917,24.830944
2868,R241,34.978,91.518194,24.844972
2869,R241,36.164,91.514083,24.855083
16183,R241,26.292,91.544194,24.773694
16185,R241,28.857,91.542083,24.796694
17247,R241,28.211,91.541778,24.790944
17248,R241,32.128,91.534361,24.823833


NameError: name 'df_roads_clean' is not defined

In [7]:
LAT_MIN, LAT_MAX = 20.5, 26.8
LON_MIN, LON_MAX = 88.0, 92.8

out_country = df[
    (df["lat"].notna()) &
    (df["lon"].notna()) &
    (
        (df["lat"] < LAT_MIN) |
        (df["lat"] > LAT_MAX) |
        (df["lon"] < LON_MIN) |
        (df["lon"] > LON_MAX)
    )
]

print("Outside Bangladesh:", len(out_country)) 


Outside Bangladesh: 51


In [22]:
#cleaned df = clean csv 
df_cleaned_roads = pd.read_csv("../data/raw/clean.csv") 
df_cleaned_roads.head()   
df_bridges.columns

#match them on road and LRP name

Index(['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition',
       'structureNr', 'roadName', 'chainage', 'width', 'constructionYear',
       'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon',
       'EstimatedLoc'],
      dtype='object')

In [21]:
df_cleaned_roads.columns

Index(['Unnamed: 0', 'road', 'chainage', 'lrp', 'lat', 'lon', 'gap', 'type',
       'name'],
      dtype='object')

In [23]:
def norm(s):
    return s.astype(str).str.strip().str.upper()

# Normalised keys
df_roads = df_cleaned_roads.copy()
df_br = df_bridges.copy()

df_roads["road_norm"] = norm(df_roads["road"])
df_roads["lrp_norm"]  = norm(df_roads["lrp"])

df_br["road_norm"] = norm(df_br["road"])
df_br["lrp_norm"]  = norm(df_br["LRPName"])

In [32]:
#df_br.describe has: 21407 rows

In [33]:
#df_roads.describe has: #52210 rows

In [ ]:
#cluster 2 bridges die niet in road file zitten 
#bridges missing in roads; #dus bridge zegt dat het op bepaalde road ligt maar code komt niet overeen met een 
#LRP code op die road 
bridge_keys = df_br[["road_norm","lrp_norm"]]
road_keys   = df_roads[["road_norm","lrp_norm"]]

bridges_missing_in_roads = bridge_keys.merge(
    road_keys, on=["road_norm","lrp_norm"], how="left", indicator=True
).query("_merge == 'left_only'").drop(columns="_merge")

print("Bridges (BMMS) missing in roads:", len(bridges_missing_in_roads))
bridges_missing_in_roads.head(20)

Bridges (BMMS) missing in roads: 6552


,road_norm,lrp_norm
0,N1,LRP001A
1,N1,LRP004B
11,N1,LRP013B
12,N1,LRP014A
15,N1,LRP018C
17,N1,LRP021C
18,N1,LRP021D
19,N1,LRP021C
21,N1,LRP023D
22,N1,LRP023C


In [ ]:
#Wat ik nu nodig heb: outlier file (groupering dus wat Yoran geinterpoleerd heeft) 
#--> die interpolatie dan aanpssen de lon en lat? in bridge file? Maar wat als ze wel goed stonden in bridge file?

In [ ]:
#ik ga ze nu overlappen 

In [39]:
#eerst ff distance functie hier zetten hebben we later nodig: 
def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000  # Straal aarde in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [ ]:
#inladen: 
# 1. Laad de SCHONE wegen (output van notebook 1)
# Pas het pad aan naar waar jij je clean.csv hebt opgeslagen
df_roads = pd.read_csv("../data/raw/clean.csv")  

# 2. Laad de RUWE bruggen
df_bridges = pd.read_excel("../data/raw/BMMS_overview.xlsx")

# Zorg dat de koppel-kolommen nummers zijn
df_roads['chainage'] = pd.to_numeric(df_roads['chainage'], errors='coerce')
df_bridges['km'] = pd.to_numeric(df_bridges['km'], errors='coerce')

# Zorg dat coordinaten nummers zijn #cooerce zorgt ervoor dat als iets niet goed staat niet verwijdert wordt maar  
#nan value wordt: dus wellicht later nog kijken!! wlke dit waren en of dit typoos waren 
for c in ['lat', 'lon']:
    df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
    df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

print(f"Wegen geladen: {len(df_roads)}")
print(f"Bruggen geladen: {len(df_bridges)}")

Wegen geladen: 52210
Bruggen geladen: 21407


In [51]:
df_analysis = df_bridges.copy()

# Nieuwe kolommen aanmaken
df_analysis['ideal_lat'] = np.nan
df_analysis['ideal_lon'] = np.nan
df_analysis['offset_meters'] = np.nan
df_analysis['error_flag'] = 'OK' # Startwaarde

# Lijst van bekende wegen
known_roads = df_analysis.loc[df_analysis['road'].isin(unique_roads), 'road'].unique()

print("Bezig met berekenen van offsets (dit duurt even)...")

for road_id in known_roads:
    # 1. Pak de weg data (Cleaned)
    road_data = df_roads[df_roads['road'] == road_id].sort_values('chainage')
    if road_data.empty: continue
        
    # 2. Pak de bruggen op deze weg
    mask = df_analysis['road'] == road_id
    bridge_kms = df_analysis.loc[mask, 'km']
    
    # 3. BEREKEN IDEALE LOCATIE (Interpolatie op chainage)
    # Dit is de "waarheid" volgens de chainage
    ideal_lats = np.interp(bridge_kms, road_data['chainage'], road_data['lat'])
    ideal_lons = np.interp(bridge_kms, road_data['chainage'], road_data['lon'])
    
    df_analysis.loc[mask, 'ideal_lat'] = ideal_lats
    df_analysis.loc[mask, 'ideal_lon'] = ideal_lons
    
    # 4. Check of brug "voorbij" de weg ligt (Clamping nodig?)
    max_len = road_data['chainage'].max()
    clamped_mask = mask & (bridge_kms > max_len)
    df_analysis.loc[clamped_mask, 'is_clamped'] = True

# --- FLAGS TOEWIJZEN ---

# Stap A: Bereken afstand (alleen als GPS bestaat)
# Haversine tussen Huidige GPS en Ideale Locatie
dists = haversine_dist(
    df_analysis['lat'], df_analysis['lon'],
    df_analysis['ideal_lat'], df_analysis['ideal_lon']
)
df_analysis['offset_meters'] = dists

# Stap B: Categorieën bepalen

# 1. Onbekende weg
df_analysis.loc[~df_analysis['road'].isin(unique_roads), 'error_flag'] = 'Unknown Road'

# 2. GPS Missing of in Afrika/Oceaan (Bounding box check)
bad_gps = (df_analysis['lat'].isna()) | (df_analysis['lat'] < 20) | (df_analysis['lat'] > 27)
df_analysis.loc[bad_gps, 'error_flag'] = 'Bad GPS (Africa/Null)'

# 3. Grote Offset (>20000m) - Waarschijnlijk verkeerde weg of slechte GPS
large_offset = (~bad_gps) & (df_analysis['offset_meters'] > 20000) #welke treshold 
df_analysis.loc[large_offset, 'error_flag'] = 'Large Offset (>20000m)'

# 4. Kleine Offset (50-500m) - Onnauwkeurigheid
small_offset = (~bad_gps) & (df_analysis['offset_meters'] >= 50) & (df_analysis['offset_meters'] <= 500)
df_analysis.loc[small_offset, 'error_flag'] = 'Small Offset (50-500m)'

# Print resultaten voor je rapport
print("\n=== DATA QUALITY DIAGNOSE ===")
print(df_analysis['error_flag'].value_counts())
print(f"\nAantal bruggen die voorbij einde weg liggen (Clamped): {df_analysis['is_clamped'].sum()}")

Bezig met berekenen van offsets (dit duurt even)...

=== DATA QUALITY DIAGNOSE ===
error_flag
OK                        12221
Small Offset (50-500m)     8739
Large Offset (>20000m)      312
Bad GPS (Africa/Null)       135
Name: count, dtype: int64

Aantal bruggen die voorbij einde weg liggen (Clamped): 411


In [52]:
import pandas as pd
import numpy as np

# 1. Laden & Schoonmaken


# Strings standaardiseren (BELANGRIJK voor matchen)
# In de road file heet de kolom 'lrp', in BMMS heet hij 'LRPName'
df_roads['road'] = df_roads['road'].astype(str).str.strip().str.upper()
df_roads['lrp'] = df_roads['lrp'].astype(str).str.strip().str.upper()

df_bridges['road'] = df_bridges['road'].astype(str).str.strip().str.upper()
df_bridges['LRPName'] = df_bridges['LRPName'].astype(str).str.strip().str.upper()

# Nummers
for c in ['chainage', 'lat', 'lon']: df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
for c in ['km', 'lat', 'lon']: df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

# --- STAP A: ONDERSCHEID MAKEN TUSSEN FOUTTYPES (Null vs Africa) ---
# We maken specifieke categorieën aan
df_bridges['gps_status'] = 'Valid' # Startwaarde

# 1. Null / NaN (Helemaal geen data)
mask_null = df_bridges['lat'].isna() | df_bridges['lon'].isna()
df_bridges.loc[mask_null, 'gps_status'] = 'Missing (NaN)'

# 2. Nul-waarden (0,0 -> ligt in de oceaan bij Afrika)
mask_zero = (df_bridges['lat'] == 0) & (df_bridges['lon'] == 0)
df_bridges.loc[mask_zero, 'gps_status'] = 'Zero Coordinates (0,0)'

# 3. Afrika / Buitenland (Wel cijfers, maar verkeerde plek)
# Alles buiten Lat 20-27 en Lon 88-93
mask_africa = (
    (df_bridges['gps_status'] == 'Valid') & 
    ((df_bridges['lat'] < 20) | (df_bridges['lat'] > 27) | 
     (df_bridges['lon'] < 88) | (df_bridges['lon'] > 93))
)
df_bridges.loc[mask_africa, 'gps_status'] = 'Out of Bounds (Africa)'

print("=== GPS KWALITEIT ANALYSE ===")
print(df_bridges['gps_status'].value_counts())


# --- STAP B: DE HYBRIDE MATCH (LRP Eerst, dan KM) ---
print("\nStart Hybride Match...")

# We gaan proberen de brug te koppelen aan de Road-data via ROAD + LRP NAAM
# We noemen de kolommen van roads even anders om verwarring te voorkomen
roads_lrp_lookup = df_roads[['road', 'lrp', 'lat', 'lon', 'chainage']].rename(
    columns={'lrp': 'LRPName', 'lat': 'road_lat', 'lon': 'road_lon', 'chainage': 'road_chainage'}
)

# Merge: We plakken de road-info aan de brug VAST als de naam matcht
# 'left' merge betekent: houd alle bruggen, zoek road info erbij
df_merged = df_bridges.merge(roads_lrp_lookup, on=['road', 'LRPName'], how='left')

# Nu hebben we bruggen MET en ZONDER match
mask_match = ~df_merged['road_lat'].isna()
print(f"Aantal bruggen gematcht op LRP Naam (Gouden standaard): {mask_match.sum()}")
print(f"Aantal bruggen NIET gematcht op naam (Terugvallen op KM): {(~mask_match).sum()}")

# --- STAP C: DE LAT/LON BEPALEN ---

df_final = df_merged.copy()
df_final['source_of_fix'] = 'None'
df_final['new_lat'] = np.nan
df_final['new_lon'] = np.nan

# 1. Voor LRP Matches: Gebruik de Road Lat/Lon (Dit is de beste data!)
df_final.loc[mask_match, 'new_lat'] = df_final.loc[mask_match, 'road_lat']
df_final.loc[mask_match, 'new_lon'] = df_final.loc[mask_match, 'road_lon']
df_final.loc[mask_match, 'source_of_fix'] = 'LRP Name Match'

# 2. Voor Niet-Matches: Gebruik Interpolatie op KM (Zoals eerder)
# Dit stukje code is iets trager omdat we moeten loopen
# We doen dit alleen voor de bruggen waar 'new_lat' nog leeg is
missing_mask = df_final['new_lat'].isna()
missing_roads = df_final.loc[missing_mask, 'road'].unique()

for r in missing_roads:
    # Pak road data
    r_data = df_roads[df_roads['road'] == r].sort_values('chainage')
    if r_data.empty: continue
    
    # Pak bruggen op deze weg die nog geen fix hebben
    sub_mask = (df_final['road'] == r) & (df_final['new_lat'].isna())
    kms = df_final.loc[sub_mask, 'km']
    
    # Interpoleer
    df_final.loc[sub_mask, 'new_lat'] = np.interp(kms, r_data['chainage'], r_data['lat'])
    df_final.loc[sub_mask, 'new_lon'] = np.interp(kms, r_data['chainage'], r_data['lon'])
    df_final.loc[sub_mask, 'source_of_fix'] = 'KM Interpolation'

# 3. Checken op verkeerde weg (Analyse vraag van jou)
# Als we een LRP Match hebben, maar de KM in BMMS wijkt enorm af van de Chainage in Roads
# Dan was de Chainage in BMMS fout (typefout in km)!
df_final['km_diff'] = abs(df_final['km'] - df_final['road_chainage'])
chainage_typos = df_final[(df_final['source_of_fix'] == 'LRP Name Match') & (df_final['km_diff'] > 1)]

print(f"\nAantal bruggen waar KM fout was, maar LRP naam ons redde: {len(chainage_typos)}")

=== GPS KWALITEIT ANALYSE ===
gps_status
Valid                     21270
Missing (NaN)                94
Out of Bounds (Africa)       26
Zero Coordinates (0,0)       17
Name: count, dtype: int64

Start Hybride Match...
Aantal bruggen gematcht op LRP Naam (Gouden standaard): 14912
Aantal bruggen NIET gematcht op naam (Terugvallen op KM): 6552

Aantal bruggen waar KM fout was, maar LRP naam ons redde: 2828


In [53]:



# Strings
df_roads['road'] = df_roads['road'].astype(str).str.strip().str.upper()
df_roads['lrp'] = df_roads['lrp'].astype(str).str.strip().str.upper()
df_bridges['road'] = df_bridges['road'].astype(str).str.strip().str.upper()
df_bridges['LRPName'] = df_bridges['LRPName'].astype(str).str.strip().str.upper()

# Nummers
for c in ['chainage', 'lat', 'lon']: df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
for c in ['km', 'lat', 'lon']: df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

print("Start Diagnose & Reparatie...")

# ---------------------------------------------------------
# STAP 1: DE DIAGNOSE (Wat is er mis met de originele data?)
# ---------------------------------------------------------
df_final = df_bridges.copy()
df_final['Original_Issue'] = 'OK' # Startwaarde

# A. Check op Null / Zero / Afrika
mask_null = df_final['lat'].isna() | df_final['lon'].isna()
df_final.loc[mask_null, 'Original_Issue'] = 'Missing Coords'

mask_zero = (df_final['lat'] == 0) & (df_final['lon'] == 0)
df_final.loc[mask_zero, 'Original_Issue'] = 'Zero Coords'

# Alles buiten Lat 20-27 en Lon 88-93 is Afrika/Oceaan
mask_africa = (
    (df_final['Original_Issue'] == 'OK') & 
    ((df_final['lat'] < 20) | (df_final['lat'] > 27) | 
     (df_final['lon'] < 88) | (df_final['lon'] > 93))
)
df_final.loc[mask_africa, 'Original_Issue'] = 'Africa/Ocean'

# B. Check op Offsets (Verkeerde weg / Slechte GPS)
# We berekenen waar de brug ZOU moeten zijn volgens KM (Interpolatie)
# Dit is puur voor de diagnose, de fix doen we straks pas definitief.
df_final['chainage_lat'] = np.nan
df_final['chainage_lon'] = np.nan

unique_roads = df_final['road'].unique()
for r in unique_roads:
    r_data = df_roads[df_roads['road'] == r].sort_values('chainage')
    if r_data.empty: continue
    
    mask = df_final['road'] == r
    kms = df_final.loc[mask, 'km']
    
    # Bereken theoretische positie
    df_final.loc[mask, 'chainage_lat'] = np.interp(kms, r_data['chainage'], r_data['lat'])
    df_final.loc[mask, 'chainage_lon'] = np.interp(kms, r_data['chainage'], r_data['lon'])

# Bereken afstand (Haversine) voor diagnose
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Alleen checken als GPS geldig is en we de weg kennen
mask_calc = (df_final['Original_Issue'] == 'OK') & (~df_final['chainage_lat'].isna())
dists = haversine(
    df_final.loc[mask_calc, 'lat'], df_final.loc[mask_calc, 'lon'],
    df_final.loc[mask_calc, 'chainage_lat'], df_final.loc[mask_calc, 'chainage_lon']
)
df_final.loc[mask_calc, 'dist_check'] = dists

# Markeer grote en kleine offsets
large_offset = (df_final['Original_Issue'] == 'OK') & (df_final['dist_check'] > 500)
df_final.loc[large_offset, 'Original_Issue'] = 'Large Offset (>500m)'

small_offset = (df_final['Original_Issue'] == 'OK') & (df_final['dist_check'] >= 50) & (df_final['dist_check'] <= 500)
df_final.loc[small_offset, 'Original_Issue'] = 'Small Offset (50-500m)'

# Check Unknown Road
mask_unknown = ~df_final['road'].isin(df_roads['road'])
df_final.loc[mask_unknown, 'Original_Issue'] = 'Unknown Road'

print("Diagnose Voltooid! Dit zijn de problemen in de originele file:")
print(df_final['Original_Issue'].value_counts())



Start Diagnose & Reparatie...
Diagnose Voltooid! Dit zijn de problemen in de originele file:
Original_Issue
Small Offset (50-500m)    8739
OK                        6452
Large Offset (>500m)      5591
Unknown Road               552
Missing Coords              30
Africa/Ocean                26
Zero Coords                 17
Name: count, dtype: int64


In [ ]:
#here we added how often the chainage is wrong, so if lon and lat are on the road but the km is wrong, as that km does not belong to that lon and lat 
#from that followed that the chainge is more wrong so that ofthen the chaina
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree



# Strings standaardiseren (Hoofdletters, geen spaties)
df_roads['road'] = df_roads['road'].astype(str).str.strip().str.upper()
df_bridges['road'] = df_bridges['road'].astype(str).str.strip().str.upper()

# Nummers forceren
for c in ['chainage', 'lat', 'lon']: df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
for c in ['km', 'lat', 'lon']: df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

# =============================================================================
# 2. BASIS DIAGNOSE (Null, Nul, Afrika)
# =============================================================================
print("Basis diagnose uitvoeren...")
df_final = df_bridges.copy()
df_final['Issue_Type'] = 'OK' # We gaan ervan uit dat het goed is, tot tegendeel bewezen

# A. Missing Coordinates
mask_null = df_final['lat'].isna() | df_final['lon'].isna()
df_final.loc[mask_null, 'Issue_Type'] = 'Missing Coords (NaN)'

# B. Zero Coordinates (0,0)
mask_zero = (df_final['lat'] == 0) & (df_final['lon'] == 0)
df_final.loc[mask_zero, 'Issue_Type'] = 'Zero Coords (0,0)'

# C. Africa / Out of Bounds
# Alles buiten Bangladesh (Lat 20-27, Lon 88-93)
mask_africa = (
    (df_final['Issue_Type'] == 'OK') & 
    ((df_final['lat'] < 20) | (df_final['lat'] > 27) | 
     (df_final['lon'] < 88) | (df_final['lon'] > 93))
)
df_final.loc[mask_africa, 'Issue_Type'] = 'Africa/Ocean (GPS Error)'

# D. Unknown Road (Weg bestaat niet in onze clean.csv)
mask_unknown = ~df_final['road'].isin(df_roads['road'])
df_final.loc[mask_unknown, 'Issue_Type'] = 'Unknown Road (Cannot Check)'

# =============================================================================
# 3. CONSISTENTIE CHECK (Chainage vs GPS)
# =============================================================================
print("Chainage vs GPS checken...")

# We berekenen waar de brug ZOU moeten zijn volgens het paaltje (KM)
df_final['ideal_lat'] = np.nan
df_final['ideal_lon'] = np.nan

# Loopje per weg (Interpolatie)
unique_roads = df_final.loc[df_final['Issue_Type'] == 'OK', 'road'].unique()

for r in unique_roads:
    r_data = df_roads[df_roads['road'] == r].sort_values('chainage')
    if r_data.empty: continue
    
    mask = (df_final['road'] == r) & (df_final['Issue_Type'] == 'OK')
    kms = df_final.loc[mask, 'km']
    
    df_final.loc[mask, 'ideal_lat'] = np.interp(kms, r_data['chainage'], r_data['lat'])
    df_final.loc[mask, 'ideal_lon'] = np.interp(kms, r_data['chainage'], r_data['lon'])

# Haversine Functie (Afstand in meters)
def haversine_np(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Bereken afstand tussen GPS en Chainage-Punt
mask_calc = (df_final['Issue_Type'] == 'OK') & (~df_final['ideal_lat'].isna())
dists = haversine_np(
    df_final.loc[mask_calc, 'lat'], df_final.loc[mask_calc, 'lon'],
    df_final.loc[mask_calc, 'ideal_lat'], df_final.loc[mask_calc, 'ideal_lon']
)
df_final.loc[mask_calc, 'dist_offset'] = dists

# E. Markeer Offsets
# Kleine offset (50m - 500m) -> Waarschijnlijk GPS onnauwkeurigheid
mask_small = (df_final['Issue_Type'] == 'OK') & (df_final['dist_offset'] >= 50) & (df_final['dist_offset'] <= 500)
df_final.loc[mask_small, 'Issue_Type'] = 'Small Offset (50-500m)'

# Grote offset (> 500m) -> Hier gaan we dieper op in!
mask_large = (df_final['Issue_Type'] == 'OK') & (df_final['dist_offset'] > 500)
df_final.loc[mask_large, 'Issue_Type'] = 'Large Offset (>500m)'

# =============================================================================
# 4. GEAVANCEERDE ANALYSE (Waarom is de offset groot?)
# =============================================================================
print("Geavanceerde analyse voor grote afwijkingen...")

# We pakken alleen de probleemgevallen
problem_bridges = df_final[df_final['Issue_Type'] == 'Large Offset (>500m)'].copy()

if not problem_bridges.empty:
    # 1. Bouw een zoekboom van ALLE punten in het wegennetwerk
    # (Dit stelt ons in staat om razendsnel de dichtstbijzijnde weg te vinden)
    valid_road_points = df_roads.dropna(subset=['lat', 'lon'])
    road_coords = valid_road_points[['lat', 'lon']].values
    road_names = valid_road_points['road'].values
    
    tree = cKDTree(road_coords)
    
    # 2. Zoek voor elke probleembrug het dichtstbijzijnde punt in HEEL Bangladesh
    bridge_coords = problem_bridges[['lat', 'lon']].values
    dists_deg, indices = tree.query(bridge_coords)
    
    # Afstand omrekenen naar meters (1 graad lat is ong 111km)
    dists_meters = dists_deg * 111000
    nearest_roads = road_names[indices]
    
    # 3. Classificeren
    # F. Wrong Chainage: Brug ligt wél dichtbij (<200m) zijn EIGEN weg, maar chainage klopt niet
    mask_wrong_km = (dists_meters < 200) & (nearest_roads == problem_bridges['road'])
    
    # G. Likely Wrong Road: Brug ligt dichtbij (<200m) een ANDERE weg
    mask_wrong_road = (dists_meters < 200) & (nearest_roads != problem_bridges['road'])
    
    # H. Nowhere: Brug ligt nergens in de buurt (>200m) van welke weg dan ook
    mask_nowhere = (dists_meters >= 200)
    
    # Update de labels in de hoofd-tabel
    # We gebruiken de index om de juiste rijen te updaten
    df_final.loc[problem_bridges[mask_wrong_km].index, 'Issue_Type'] = 'Wrong Chainage (On Road, but KM mismatch)'
    df_final.loc[problem_bridges[mask_wrong_road].index, 'Issue_Type'] = 'Likely Wrong Road (Near other road)'
    df_final.loc[problem_bridges[mask_nowhere].index, 'Issue_Type'] = 'GPS Error (Nowhere near any road)'

# =============================================================================
# 5. RESULTAAT
# =============================================================================
print("\n=== COMPLETE DATA QUALITY ANALYSE ===")
print(df_final['Issue_Type'].value_counts())

# Optioneel: Opslaan om in Excel te bekijken
# df_final.to_csv('bridges_diagnosed.csv', index=False)

Basis diagnose uitvoeren...
Chainage vs GPS checken...
Geavanceerde analyse voor grote afwijkingen...

=== COMPLETE DATA QUALITY ANALYSE ===
Issue_Type
Small Offset (50-500m)                       8739
OK                                           6452
Wrong Chainage (On Road, but KM mismatch)    3909
GPS Error (Nowhere near any road)            1459
Unknown Road (Cannot Check)                   552
Likely Wrong Road (Near other road)           223
Missing Coords (NaN)                           30
Africa/Ocean (GPS Error)                       26
Zero Coords (0,0)                              17
Name: count, dtype: int64


In [58]:

# ---------------------------------------------------------
# STAP 2: DE REPARATIE (Hybride)
# ---------------------------------------------------------
df_final['new_lat'] = np.nan
df_final['new_lon'] = np.nan
df_final['fix_method'] = 'None'

# A. LRP Match (Goud) - Gebruik de naam
# We merge met de road file op Road + LRPNaam
roads_lookup = df_roads[['road', 'lrp', 'lat', 'lon']].rename(columns={'lrp': 'LRPName', 'lat': 'rlat', 'lon': 'rlon'})
df_merged = df_final.merge(roads_lookup, on=['road', 'LRPName'], how='left')

# Waar we een match hebben, nemen we die over
mask_lrp = ~df_merged['rlat'].isna()
# Let op: de index van df_merged is misschien anders, dus we gebruiken de values
# (Of we updaten df_final direct als we zeker weten dat de volgorde klopt, merge kan volgorde veranderen)
# Veiliger: Update df_final met een map, of werk verder in df_merged.
# We werken verder in df_merged omdat dat makkelijker is
df_final = df_merged.copy() 

# Zet LRP matches
mask_lrp = ~df_final['rlat'].isna()
df_final.loc[mask_lrp, 'new_lat'] = df_final.loc[mask_lrp, 'rlat']
df_final.loc[mask_lrp, 'new_lon'] = df_final.loc[mask_lrp, 'rlon']
df_final.loc[mask_lrp, 'fix_method'] = 'LRP Match'

# B. Chainage Interpolatie (Zilver) - Voor de rest
# We hebben 'chainage_lat' al berekend in Stap 1! Dus die gebruiken we.
mask_needs_fix = df_final['new_lat'].isna() & (~df_final['chainage_lat'].isna())

df_final.loc[mask_needs_fix, 'new_lat'] = df_final.loc[mask_needs_fix, 'chainage_lat']
df_final.loc[mask_needs_fix, 'new_lon'] = df_final.loc[mask_needs_fix, 'chainage_lon']
df_final.loc[mask_needs_fix, 'fix_method'] = 'Chainage Interpolation'


# ---------------------------------------------------------
# STAP 3: OPSLAAN
# ---------------------------------------------------------
# Overschrijf de oude lat/lon (behalve bij unknown road)
has_fix = ~df_final['new_lat'].isna()
df_final.loc[has_fix, 'lat'] = df_final.loc[has_fix, 'new_lat']
df_final.loc[has_fix, 'lon'] = df_final.loc[has_fix, 'new_lon']

# Selecteer kolommen voor export
cols = ['road', 'km', 'type', 'LRPName', 'lat', 'lon', 'Original_Issue', 'fix_method']
df_final[cols].to_csv('bridges_cleaned_final.csv', index=False)

print("\nKlaar! Opgeslagen met diagnose en reparatie kolommen.")
print("Overzicht van fixes:")
print(df_final.groupby(['Original_Issue', 'fix_method']).size())

KeyError: 'rlat'